In [ ]:
cd ~/workspace/mmf_nr/

In [ ]:
import skimage.io
import os
from glob import glob
from tqdm import tqdm
import warnings
import numpy as np

In [ ]:
inpainting_src_dirs = 'save/external_inpainting_synsin_realestate10k/realestate10k_nodepth_perceptual_l1laplacian_inpaintGonly_scale2_lowL1/50000/realestate10k_inpainting_{}/inpainting_src/'
inpainting_mask_dirs = 'save/external_inpainting_synsin_realestate10k/realestate10k_nodepth_perceptual_l1laplacian_inpaintGonly_scale2_lowL1/50000/realestate10k_inpainting_{}/inpainting_mask/'

splits = ['train_dup2', 'val', 'test']

In [ ]:
def extract_mask_for_dir(inpainting_src_dir, inpainting_mask_dir):
    src_png_files = sorted(glob(os.path.join(inpainting_src_dir, "*.png")))
    os.makedirs(inpainting_mask_dir, exist_ok=True)
    warnings.filterwarnings('ignore', category=UserWarning)  # ignore warning on low-constract images
    for src_file in tqdm(src_png_files):
        assert src_file.startswith(inpainting_src_dir)
        mask_file = src_file.replace(inpainting_src_dir, inpainting_mask_dir)

        im = skimage.io.imread(src_file)
        mask = im[..., 3]
        assert mask.dtype == np.uint8
        # flip the mask: Rethinking-Inpainting-MEDFE needs 1 on regions to inpaint
        # which is the opposite to our alpha_mask from mesh rendering
        mask = 255 - mask
        skimage.io.imsave(mask_file, mask)

In [ ]:
for split in splits:
    print('processing split', split)
    extract_mask_for_dir(
        inpainting_src_dirs.format(split), inpainting_mask_dirs.format(split)
    )